In [18]:
import multiprocessing as mp
import queue, time

class TimeKeeper(mp.Process):
    
    def __init__(self, resolution=0.01):
        super(TimeKeeper, self).__init__()
        self.resolution = resolution
        self.queue = mp.Queue()
        self.daemon = True
        
    def run(self):
        f = open('timekeeper_output.txt','w')
        t0 = time.time()
        
        cont = True
        while cont:
            
            try:
                external_data = self.queue.get(block=False)
            except queue.Empty:
                external_data = None
            
            if external_data:
                f.write('{}\n'.format(external_data))
            if time.time()-t0 >= self.resolution:
                f.write('{:0.3f}\n'.format(time.time()))
                t0 = time.time()
            
            if external_data == 'stop':
                break
            
        f.close()

In [19]:
import time, random

tk = TimeKeeper()
tk.start()

t0 = time.time()
while time.time()-t0 < 5:
    if (time.time()-t0)%1 == 0:
        tk.queue.put(random.choice('abcdefg'))
tk.queue.put('stop')